In [1]:
import numpy as np
import cupy as cp
from cupyx.scipy.special import ndtr

cp.random.seed(412)

In [2]:
def reinforce_function(func, n, *args):
    """ Repeat a function with certain parameters several times and return the
    mean result

    Parameters
    ----------
    func : function
        The function to be repeated
    n : int
        The repeating time
    args : tuple
        The parameters for the function

    Returns
    -------
    mean_value : float
        The mean value of n times running
    """

    value = []
    for i in range(n):
        value.append(func(*args))
    value = cp.array(value)
    mean_value = cp.mean(value, axis=0).item()
    
    return mean_value


# Problem 1

In [3]:
def basket_cds(n, s):
    """

    Parameters
    ----------
    n : int or float
        The simulation size
    s : float
        The annual protection payment for the CDS

    Returns
    -------
    v : float
        The simulated present value of the CDS

    """

    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the contract and assets
    r, t = 0.03, 5
    default_int = cp.array([0.05, 0.01, 0.05, 0.05, 0.01, 0.1, 0.01, 0.09,
                            0.1, 0.02]).reshape(10, -1)
    recov_rate = cp.array([0.1, 0.1, 0.3, 0.1, 0.3, 0.1, 0.2, 0.2,
                           0.1, 0.1]).reshape(10, -1)
    # Define parameters for the multivariate normal distribution
    cov = cp.full((10, 10), 0.2)
    cp.fill_diagonal(cov, 1)
    # Use the Cholesky decomposition to derive the transformation matrix
    a = cp.linalg.cholesky(cov)
    # Generate the correlated normal random variables
    z = cp.random.normal(size=(10, n))
    x = cp.dot(a, z)
    # Use the antithetic method to reduce the variance
    x = cp.hstack((x, -x))
    # Calculate the default time
    tau = -cp.log(1 - ndtr(x)) / default_int.reshape((10, 1))
    # Select samples with the fifth default time less than 5 years
    tau_rank_ind = cp.argsort(tau, axis=0)
    tau_5 = tau[tau_rank_ind[4, :], range(2 * n)]
    tau_5_bool = tau_5 <= t
    # Calculate the possible income for CDS holders
    v_value = cp.zeros(2 * n)
    v_value[tau_5_bool] = ((1 - recov_rate[tau_rank_ind[4, tau_5_bool]]).T *
                           cp.exp(-r * tau_5[tau_5_bool])).ravel()
    # Calculate the discounted payments
    payments = [s * cp.exp(-r * (i + 1)).get() for i in range(t)]
    cum_payments = cp.cumsum(cp.array([0] + payments))
    payments = cp.array(payments)
    # Find the last complete payment when the CDS is triggered
    frac_t, comp_t = cp.modf(tau_5[tau_5_bool])
    comp_t = comp_t.astype('int')
    # Calculate the outcome for CDS holders
    v_port = cp.full(2 * n, cp.sum(payments))
    v_port[tau_5_bool] = cum_payments[comp_t] + payments[comp_t] * frac_t * \
                         cp.exp(-r * tau_5[tau_5_bool])
    # Calculate the CDS's value
    v = v_value - v_port
    ev = cp.mean(v)

    return ev


In [4]:
# Output the results
print("The fair value of the CDS is ${:.4f}.".format(
    reinforce_function(basket_cds, 100, 1e6, .01421)))

The fair value of the CDS is $0.0000.


# Problem 2

In [5]:
def startup_valuation(n, x0):
    # Convert the simulation size into an integer
    n = int(n)
    # Initialize parameters of the asset
    pa, pb, ia, ib = 450, 1000, 50, 100
    r, sigma = 0.025, 0.9
    # Generate random variables
    u = cp.random.uniform(size=n)
    t = 6 * cp.sqrt(u)
    z = cp.random.normal(size=n)
    # Calculate the exit value
    xt = x0 * cp.exp((r - sigma ** 2 / 2) * t + sigma * cp.sqrt(t) * z)
    # Calculate the payout to Series B investors
    outcome = cp.where(xt < 1000,
                       cp.maximum(cp.minimum((ib / (ia + ib)) * xt, ib),
                                  xt * ib / pb),
                       xt * ib / pb)
    eo = cp.mean(cp.exp(-r * t) * outcome).item()

    return eo


In [6]:
# Output the results
print("The fair investment for Series B investor is ${:.2f}.".format(
    reinforce_function(startup_valuation, 1000, 1e7, 672.52)))
print("The overvaluation is {:.2%}.".format((1000 - 672.5) / 672.5))

The fair investment for Series B investor is $100.00.
The overvaluation is 48.70%.
